In [1]:
import math
import os
import gc
import re
import sys
import time
import json
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from scipy.optimize import leastsq
from sklearn.linear_model import LinearRegression
from scipy import sparse
from xpinyin import Pinyin
import numpy as np

## 一.车系匹配

### 1.力洋车型库处理

In [51]:
def process_brand_name(df):
    """
    删除没用的字符
    """
    text = df['brand_name']
    text = text.replace('MINI(迷你)', 'MINI')
    text = text.replace('RUF(如虎)', '如虎')
    text = text.replace('Pagani', '帕加尼')
    text = text.replace('柯尼赛格', '科尼赛克')
    text = text.replace('布嘉迪', '布加迪')
    text = text.replace('幻速', '北汽幻速')
    text = text.replace('昌河', '北汽昌河')
    text = text.replace('汽车', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

def process_model_name(df):
    """
    删除没用的字符
    """
    text = df['model_name']
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('（', '')
    text = text.replace('）', '')
    text = text.replace('-', '')
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('/', '')
    text = text.replace('°', '')
    text = text.replace('!', '')
    text = text.lower()
    text = text.replace('ⅲ', 'iii')
    text = text.replace('pagani', '帕加尼')
    text = text.replace('柯尼赛格', '科尼赛克')
    text = text.replace('科尼赛格', '科尼赛克')
    text = text.replace('布嘉迪', '布加迪')
    text = text.replace('吉利康迪全球鹰', '全球鹰')
    text = text.replace('汽车', '')
    if df['brand_name'] == '长丰猎豹':
        text = text.replace('猎豹', '长丰猎豹')
    if df['brand_name'] == '北汽制造':
        text = text.replace('北汽', '')
    if df['brand_name'] == '北汽绅宝':
        text = text.replace('北京绅宝', '北汽绅宝')
    if df['brand_name'] == '北汽幻速':
        text = text.replace('幻速', '北汽幻速')
    if df['brand_name'] == '北汽昌河':
        text = text.replace('昌河', '北汽昌河')
    text = text.replace(' ', '')
    # text = text.lower()
    return text

def process_final_text(df):
    """
    删除没用的字符
    """
    text = df['model_name'].replace(df['brand_name'], '')
    # text = text.lower()
    return df['brand_name']+text

In [52]:
open_category = pd.read_csv('../tmp/train/open_category.csv')
open_category = open_category.loc[~(open_category['status'] == 'D'), :].reset_index(drop=True)

# 人工解决的车系问题
open_category.loc[(open_category['id'] == 204), 'name'] = '奔驰A级(进口)'
open_category = open_category.loc[~(open_category['id'].isin([1069,1264])), :] 

liyang_brand = open_category.loc[(open_category['parent'].isnull()), ['id', 'name', 'slug']].rename(columns={'name': 'brand_name', 'slug': 'parent'}).reset_index(drop=True)
liyang_model = open_category.loc[(open_category['parent'].notnull()), ['id', 'name', 'slug', 'parent']].rename(columns={'name': 'model_name',
             'slug': 'model_slug', 'id': 'model_id'}).reset_index(drop=True)
liyang_model = liyang_model.merge(liyang_brand, how='left', on=['parent']).rename(columns={'id': 'brand_id'})
liyang_model = liyang_model.loc[:, ['brand_id', 'model_id', 'brand_name', 'model_name', 'model_slug']]

liyang_model['brand_name'] = liyang_model.apply(process_brand_name, axis=1)
liyang_model['model_name'] = liyang_model.apply(process_model_name, axis=1)
liyang_model['final_text'] = liyang_model.apply(process_final_text, axis=1)
liyang_model = liyang_model.sort_values(by=['brand_name', 'model_name']).reset_index(drop=True)
print('力洋车系数量:',len(liyang_model))

verify = liyang_model.groupby(['final_text'])['model_slug'].count().reset_index().rename(columns={'model_slug':'num'})
print('车系名称重复的车系,请人工检查:')
print(verify.loc[(verify['num'] > 1),:])
liyang_model.to_csv('../tmp/train/man_liyang.csv', index=False)

力洋车系数量: 1763
车系名称重复的车系,请人工检查:
Empty DataFrame
Columns: [final_text, num]
Index: []


### 2.汽车之家车型库处理

In [59]:
def process_brand_name(df):
    """
    删除没用的字符
    """
    text = df['brand_name']

    text = text.replace('Jeep', '吉普')
    text = text.replace('SWM斯威汽车', '斯威')
    text = text.replace('名爵', 'MG')
    text = text.replace('上汽大通', '大通')
    text = text.replace('福汽启腾', '启腾')
    text = text.replace('汽车', '')
    text = text.replace('・', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

def process_model_name(df):
    """
    删除没用的字符
    """
    text = df['model_name']
    text = text.lower()
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('（', '')
    text = text.replace('）', '')
    text = text.replace('-', '')
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('/', '')
    text = text.replace('°', '')
    text = text.replace('!', '')
    text = text.replace('名爵', 'mg')
    text = text.replace('上汽大通', '大通')
    text = text.replace('风行', '东风风行')
    text = text.replace('swm斯威', '斯威')
    if df['brand_name'] == '北汽制造':
        text = text.replace('北汽', '')
    if df['brand_name'] == '北汽绅宝':
        text = text.replace('绅宝', '北汽绅宝')
    text = text.replace('汽车', '')
    text = text.replace(' ', '')

    return text

def process_final_text(df):
    """
    删除没用的字符
    """
    text = df['model_name'].replace(df['brand_name'], '')
    # text = text.lower()
    return df['brand_name']+text

In [60]:
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_model = car_autohome_all.loc[:, ['brand_slug', 'model_slug', 'brand_name', 'model_name']]
car_autohome_model = car_autohome_model.drop_duplicates(['brand_slug', 'model_slug']).reset_index(drop=True)
car_autohome_model['brand_name'] = car_autohome_model.apply(process_brand_name, axis=1)
car_autohome_model['model_name'] = car_autohome_model.apply(process_model_name, axis=1)
car_autohome_model['final_text'] = car_autohome_model.apply(process_final_text, axis=1)
car_autohome_model = car_autohome_model.sort_values(by=['brand_name', 'model_name']).reset_index(drop=True)
car_autohome_model = car_autohome_model.rename(columns={'brand_slug':'car_autohome_brand_id','model_slug':'car_autohome_model_id'})
print('汽车之家车系数量:',len(car_autohome_model))

verify = car_autohome_model.groupby(['final_text'])['car_autohome_model_id'].count().reset_index().rename(columns={'car_autohome_model_id':'num'})
print('车系名称重复的车系,请人工检查:')
print(verify.loc[(verify['num'] > 1),:])

car_autohome_model.to_csv('../tmp/train/man_car_autohome.csv', index=False)

汽车之家车系数量: 1826
车系名称重复的车系,请人工检查:
Empty DataFrame
Columns: [final_text, num]
Index: []


In [105]:
def full_include_match(df, car_autohome):
    if str(df['car_autohome_model_id']) != 'nan':
        return pd.Series([df['car_autohome_brand_id'], df['car_autohome_model_id']])
    else:
        match_num = 0
        for i in range(0,len(car_autohome)):
            liyang_text = set(df['final_text'])
            car_autohome_text = set(car_autohome.loc[i, 'final_text'])
            intersection = liyang_text.intersection(car_autohome_text)
            if (intersection == liyang_text) or (intersection == car_autohome_text):
                if len(intersection) > match_num:        
                    match_num = len(intersection)
                    brand_temp = car_autohome.loc[i, 'car_autohome_brand_id']
                    model_temp = car_autohome.loc[i, 'car_autohome_model_id']
        if match_num > 0:
            return pd.Series([brand_temp, model_temp])
    return pd.Series([np.NAN, np.NAN])

liyang_model_temp = liyang_model.copy()
car_autohome_model_temp = car_autohome_model.loc[:, ['car_autohome_brand_id','car_autohome_model_id','final_text']]
model_match = liyang_model_temp.merge(car_autohome_model_temp, how='left', on=['final_text'])
car_autohome_data = car_autohome_model_temp.loc[~(car_autohome_model_temp['car_autohome_model_id'].isin(list(model_match.car_autohome_model_id.values))),:].reset_index(drop=True)
model_match[['car_autohome_brand_id','car_autohome_model_id']] = model_match.apply(full_include_match, args=(car_autohome_data,), axis=1)

verify = model_match.groupby(['car_autohome_model_id'])['model_id'].count().reset_index().rename(columns={'model_id':'num'})
verify = verify.loc[(verify['num'] > 1),:]

model_match.loc[(model_match['car_autohome_model_id'].isin(list(verify.car_autohome_model_id.values))), ['car_autohome_brand_id','car_autohome_model_id']] = pd.Series([np.NAN,np.NAN])
model_match.to_csv('../tmp/train/final_liyang_model.csv', index=False)
model_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1763 entries, 0 to 1762
Data columns (total 8 columns):
brand_id                 1763 non-null int64
model_id                 1763 non-null int64
brand_name               1763 non-null object
model_name               1763 non-null object
model_slug               1763 non-null object
final_text               1763 non-null object
car_autohome_brand_id    1528 non-null float64
car_autohome_model_id    1528 non-null float64
dtypes: float64(2), int64(2), object(4)
memory usage: 124.0+ KB


In [163]:
model_match = pd.read_csv('../tmp/train/final_liyang_model.csv')

verify = model_match.groupby(['car_autohome_model_id'])['model_id'].count().reset_index().rename(columns={'model_id':'num'})
print('确认是否有重复匹配,请人工检查:')
print(verify.loc[(verify['num'] > 1),:])

print(model_match.info())

确认是否有重复匹配,请人工检查:
Empty DataFrame
Columns: [car_autohome_model_id, num]
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1763 entries, 0 to 1762
Data columns (total 8 columns):
brand_id                 1763 non-null int64
model_id                 1763 non-null int64
brand_name               1763 non-null object
model_name               1763 non-null object
model_slug               1763 non-null object
final_text               1763 non-null object
car_autohome_brand_id    1550 non-null float64
car_autohome_model_id    1550 non-null float64
dtypes: float64(2), int64(2), object(4)
memory usage: 110.3+ KB
None


## 二.款型匹配

In [164]:
def process_year(df):
    if len(re.findall(r'(\d+)款',df['detail_model'])) > 0:
        return int(re.findall(r'(\d+)款',df['detail_model'])[0])
    else:
        return int(re.findall(r'\d\d\d\d',df['detail_model'])[0])

open_model_detail = pd.read_csv('../tmp/train/open_model_detail.csv', low_memory=False)
open_model_detail['year'] = open_model_detail.apply(process_year, axis=1)
open_model_detail = open_model_detail.rename(columns={'global_slug':'model_slug','year':'online_year'})
open_model_detail = open_model_detail.loc[(open_model_detail['price_bn'] > 0), :].reset_index(drop=True)
final_model = pd.read_csv('../tmp/train/final_liyang_model.csv')
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')

final_model = final_model.loc[(final_model['car_autohome_model_id'].notnull()), :].reset_index(drop=True)
gpj_match_model = open_model_detail.loc[(open_model_detail['model_slug'].isin(list(set(final_model.model_slug.values)))), :]
gpj_match_model = gpj_match_model.merge(final_model.loc[:,['model_slug','car_autohome_brand_id','car_autohome_model_id']],how='left',on=['model_slug'])
print('力洋总款型数:', len(open_model_detail))
print('力洋匹配的车系量:', len(final_model))
print('力洋匹配上车系的款型量:', len(gpj_match_model))

car_autohome_all = car_autohome_all.drop(['manufacturer','body','energy','volume_extend','volume','emission_standard','listed_year','control'], axis=1)
print('汽车之家车系总量:',len(list(set(car_autohome_all.model_slug.values))))

# 汽车之家补充的车系
supplement_model = car_autohome_all.loc[~(car_autohome_all['model_slug'].isin(list(set(final_model.car_autohome_model_id.values)))), :].reset_index(drop=True)
match_model = car_autohome_all.loc[(car_autohome_all['model_slug'].isin(list(set(final_model.car_autohome_model_id.values)))), :].reset_index(drop=True)
# match_model = match_model.rename(columns={'model_slug':'car_autohome_model_id','detail_slug':'car_autohome_detail_id'}).loc[:, ['car_autohome_model_id','car_autohome_detail_id','detail_name','price_bn','online_year']]
print('汽车之家补充的车系量:', len(list(set(supplement_model.model_slug.values))))
print('汽车之家匹配的车系量:', len(list(set(match_model.model_slug.values))))
print('汽车之家匹配的款型量:', len(list(set(match_model.detail_slug.values))))
# final_detail = gpj_match_model.merge(match_model,how='left',on=['car_autohome_model_id','online_year','price_bn'])

力洋总款型数: 38668
力洋匹配的车系量: 1550
力洋匹配上车系的款型量: 36511
汽车之家车系总量: 1826
汽车之家补充的车系量: 276
汽车之家匹配的车系量: 1550
汽车之家匹配的款型量: 28960


In [165]:
def cal_div(df, car_autohome_temp):
    car_autohome_temp['price_bn_div'] = abs(df['price_bn'] - car_autohome_temp['price_bn'])
    temp = car_autohome_temp.loc[(car_autohome_temp['price_bn_div'] == min(list(car_autohome_temp.price_bn_div.values))),:].reset_index(drop=True)
    return pd.Series(list(temp.loc[0, ['detail_slug','detail_name','price_bn']].values))

model_year = gpj_match_model.loc[:, ['car_autohome_model_id','online_year']].drop_duplicates(['car_autohome_model_id','online_year'])

result_model_year = pd.DataFrame()
no_have_model_year = pd.DataFrame()
except_model_year = pd.DataFrame()
count = 0
for car_autohome_model_id,online_year in model_year.loc[:, ['car_autohome_model_id','online_year']].values:
    count = count+1
    gpj_temp = gpj_match_model.loc[(gpj_match_model['car_autohome_model_id'] == car_autohome_model_id)&(gpj_match_model['online_year'] == online_year), :].reset_index(drop=True)
    gpj_temp['car_autohome_detail_id'] = np.NAN
    gpj_temp['detail_name'] = np.NAN
    gpj_temp['car_autohome_price_bn'] = np.NAN
    car_autohome_temp = match_model.loc[(match_model['model_slug'] == car_autohome_model_id)&(match_model['online_year'] == online_year), :].reset_index(drop=True)
    if len(car_autohome_temp) == 0:
        no_have_model_year = no_have_model_year.append(gpj_temp, sort=False).reset_index(drop=True)
        continue
    gpj_temp[['car_autohome_detail_id','detail_name','car_autohome_price_bn']] = gpj_temp.apply(cal_div,args=(car_autohome_temp,),axis=1)
    gpj_temp['price_bn_rate'] = abs((gpj_temp['price_bn'] - gpj_temp['car_autohome_price_bn']) / gpj_temp['car_autohome_price_bn'])
    gpj_temp = gpj_temp.loc[(gpj_temp['price_bn_rate'] <= 0.05), :].reset_index(drop=True)
    if (len(gpj_temp)/len(car_autohome_temp)) >= 0.8:
        result_model_year = result_model_year.append(gpj_temp, sort=False).reset_index(drop=True)
    else:
        except_model_year = except_model_year.append(car_autohome_temp, sort=False).reset_index(drop=True)

result_model_year = result_model_year.drop(['car_autohome_brand_id','detail_name','car_autohome_price_bn','price_bn_rate'], axis=1)
except_model_year = except_model_year.drop(['price_bn_div'], axis=1)
print('使用力洋款型总量:',len(result_model_year))
print('使用汽车之家款型总量:',len(except_model_year))
print('力洋未匹配上车系年份款型总量:',len(no_have_model_year))

使用力洋款型总量: 26785
使用汽车之家款型总量: 3030
力洋未匹配上车系年份款型总量: 5927


## 四.品牌车系款型组装

In [166]:
def process_year(df):
    if len(re.findall(r'(\d+)款',df['detail_model'])) > 0:
        return int(re.findall(r'(\d+)款',df['detail_model'])[0])
    else:
        return int(re.findall(r'\d\d\d\d',df['detail_model'])[0])

open_category = pd.read_csv('../tmp/train/open_category.csv', low_memory=False)

# 人工解决的车系问题
open_category = open_category.loc[~(open_category['status'] == 'D'), :].reset_index(drop=True)
open_category.loc[(open_category['id'] == 204), 'name'] = '奔驰A级(进口)'
open_category = open_category.loc[~(open_category['id'].isin([1069,1264])), :] 

gpj_brand = open_category.loc[(open_category['parent'].isnull()), :].reset_index(drop=True)
gpj_model = open_category.loc[(open_category['parent'].notnull()), :].reset_index(drop=True)

gpj_detail = pd.read_csv('../tmp/train/open_model_detail.csv', low_memory=False)
gpj_detail['year'] = gpj_detail.apply(process_year, axis=1)
gpj_detail = gpj_detail.rename(columns={'global_slug':'model_slug','year':'online_year'})
gpj_detail = gpj_detail.loc[(gpj_detail['price_bn'] > 0), :].reset_index(drop=True)

final_model = pd.read_csv('../tmp/train/final_liyang_model.csv')
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')

In [167]:
# 补充品牌
def process_brand(df, column_name):
    return 'car_autohome_brand_'+str(df[column_name])

def process_model(df, column_name):
    return 'car_autohome_model_'+str(df[column_name])

def process_detail(df, column_name):
    return 'car_autohome_detail_'+str(df[column_name])

def process_keywords(df):
    p = Pinyin()
    keywords = p.get_pinyin(df['name'], '')
    final_keywords = ','+keywords+','+df['name']+','+keywords
    first_letter = p.get_initial(df['name'][0])
    return pd.Series([final_keywords,first_letter,keywords])

liyang_brand = final_model.loc[(final_model['car_autohome_brand_id'].notnull()), :].reset_index(drop=True)
supplement_brand = list(set(supplement_model.brand_slug.values).difference(set(liyang_brand.car_autohome_brand_id.values)))
supplement_brand = car_autohome_all.loc[(car_autohome_all['brand_slug'].isin(supplement_brand)), ['brand_slug','brand_name']]
supplement_brand = supplement_brand.drop_duplicates(['brand_slug','brand_name']).reset_index(drop=True)
supplement_brand = supplement_brand.rename(columns={'brand_name':'name','brand_slug':'slug'})
supplement_brand['car_autohome_brand_id'] = supplement_brand['slug']
supplement_brand['slug'] = supplement_brand.apply(process_brand,args=('slug',), axis=1)
supplement_brand[['keywords', 'first_letter', 'pinyin']] = supplement_brand.apply(process_keywords, axis=1)
supplement_brand = supplement_brand.drop(['car_autohome_brand_id'], axis=1)

gpj_brand = gpj_brand.loc[(gpj_brand['id'].isin(list(set(liyang_brand.brand_id.values)))), :].reset_index(drop=True)
gpj_brand = gpj_brand.append(supplement_brand, sort=False).reset_index(drop=True)
gpj_brand['has_detailmodel'] = 1
gpj_brand['units'] = 0
gpj_brand['score'] = 0
gpj_brand['status'] = 'Y'
gpj_brand['popular'] = 'B'
gpj_brand['on_sale'] = '1'
gpj_brand.to_csv('../tmp/train/combine_brand.csv', index=False)
print('补充品牌数量:', len(supplement_brand))
print('力洋匹配上品牌数量:', len(list(set(liyang_brand.brand_id.values))))
print('汽车之家品牌数量:', len(list(set(car_autohome_all.brand_slug.values))))
print('最终组合品牌数量:', len(gpj_brand))

补充品牌数量: 37
力洋匹配上品牌数量: 141
汽车之家品牌数量: 196
最终组合品牌数量: 178


In [168]:
# 补充车系
def process_brand_step2(df, final_model, gpj_brand):
    brand_id = final_model.loc[(final_model['car_autohome_brand_id'] == df['car_autohome_brand_id']), 'brand_id'].values
    if len(brand_id) == 0:
        return 'car_autohome_brand_'+str(df['car_autohome_brand_id'])
    slug = gpj_brand.loc[(gpj_brand['id'] == brand_id[0]), 'slug'].values[0]
    return slug

# final_model = final_model.drop(['car_autohome_brand_id'],axis=1)
# temp_copy = final_brand.loc[:, ['id','car_autohome_brand_id']].rename(columns={'id':'brand_id'})
# final_model = final_model.merge(temp_copy,how='left',on=['brand_id'])
liyang_model = final_model.loc[(final_model['car_autohome_model_id'].notnull()), :].reset_index(drop=True)
gpj_model = gpj_model.loc[(gpj_model['id'].isin(list(set(liyang_model.model_id.values)))),:].reset_index(drop=True)
temp_copy = final_model.loc[:, ['model_slug','car_autohome_model_id']].rename(columns={'model_slug':'slug'})
gpj_model = gpj_model.merge(temp_copy, how='left', on=['slug'])

supplement_model_temp = supplement_model.loc[:, ['brand_slug','model_name','model_slug']]
supplement_model_temp = supplement_model_temp.drop_duplicates(['brand_slug','model_name','model_slug'])
supplement_model_temp = supplement_model_temp.rename(columns={'brand_slug':'parent', 'model_name':'name', 'model_slug':'slug'})
supplement_model_temp['car_autohome_model_id'] = supplement_model_temp['slug']
supplement_model_temp['slug'] = supplement_model_temp.apply(process_model,args=('slug',), axis=1)
supplement_model_temp[['keywords', 'first_letter', 'pinyin']] = supplement_model_temp.apply(process_keywords, axis=1)
supplement_model_temp = supplement_model_temp.drop(['first_letter'], axis=1)
merge = car_autohome_all.loc[:, ['model_slug', 'body', 'manufacturer']].drop_duplicates(['model_slug', 'body', 'manufacturer']).rename(columns={'model_slug':'car_autohome_model_id','body':'classified','manufacturer':'mum'}).reset_index(drop=True)
supplement_model_temp = supplement_model_temp.merge(merge, how='left', on=['car_autohome_model_id'])
supplement_model_temp['normalized_name'] = supplement_model_temp['name']

gpj_model = gpj_model.append(supplement_model_temp, sort=False).reset_index(drop=True)
car_autohome_model = car_autohome_all.loc[:,['brand_slug','model_slug']].drop_duplicates(['brand_slug','model_slug']).rename(columns={'model_slug':'car_autohome_model_id','brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
gpj_model = gpj_model.merge(car_autohome_model, how='left', on=['car_autohome_model_id'])
gpj_model['parent'] = gpj_model.apply(process_brand_step2,args=(final_model, gpj_brand), axis=1)
gpj_model = gpj_model.drop(['car_autohome_brand_id'], axis=1)
gpj_model['has_detailmodel'] = 1
gpj_model['units'] = 0
gpj_model['score'] = 0
gpj_model['status'] = 'Y'
gpj_model['popular'] = 'B'
gpj_model['on_sale'] = '1'
gpj_model.to_csv('../tmp/train/combine_model.csv', index=False)
print('补充车系数量:', len(list(set(supplement_model_temp.slug.values))))
print('力洋匹配上车系数量:', len(liyang_model))
print('汽车之家车系数量:', len(list(set(car_autohome_all.model_slug.values))))
print('最终组合车系数量:', len(gpj_model))

补充车系数量: 276
力洋匹配上车系数量: 1550
汽车之家车系数量: 1826
最终组合车系数量: 1826


In [169]:
# 补充款型
def process_model_step2(df, gpj_model, column_name):
    return gpj_model.loc[(gpj_model['car_autohome_model_id'] == df['model_slug']), 'slug'].values[0]

model_year = car_autohome_all.loc[:, ['model_slug','online_year']].drop_duplicates(['model_slug','online_year'])

supplement_detail = supplement_model.append(except_model_year, sort=False).reset_index(drop=True)
for model_slug,online_year in model_year.loc[:, ['model_slug','online_year']].values:
    gpj_temp = result_model_year.loc[(result_model_year['car_autohome_model_id'] == model_slug)&(result_model_year['online_year'] == online_year), :].reset_index(drop=True)
    car_autohome_temp = car_autohome_all.loc[(car_autohome_all['model_slug'] == model_slug)&(car_autohome_all['online_year'] == online_year), :].reset_index(drop=True)
    if len(gpj_temp) == 0:
        supplement_detail = supplement_detail.append(car_autohome_temp, sort=False).reset_index(drop=True)
        
liyang_detail = result_model_year.copy()
liyang_detail = liyang_detail.drop(['car_autohome_model_id'], axis=1)
supplement_detail = car_autohome_all.loc[(car_autohome_all['detail_slug'].isin(list(set(supplement_detail.detail_slug.values)))), :]
supplement_detail = supplement_detail.loc[:, ['model_slug','detail_slug','detail_name','price_bn','online_year','volume','control','volume_extend','emission_standard','listed_year']]
supplement_detail = supplement_detail.rename(columns={'detail_slug':'car_autohome_detail_id',
                                                      'detail_name':'detail_model'})
supplement_detail['model_slug'] = supplement_detail.apply(process_model_step2, args=(gpj_model, 'model_slug',), axis=1)
supplement_detail['model_detail_slug'] = supplement_detail.apply(process_detail, args=('car_autohome_detail_id',), axis=1)

gpj_detail = liyang_detail.append(supplement_detail, sort=False)
gpj_detail = gpj_detail.rename(columns={'model_slug':'global_slug','online_year':'year','model_detail_slug':'detail_model_slug'})
gpj_detail['created_on'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
gpj_detail['status'] = 'Y'
gpj_detail['has_param'] = 'Y'
gpj_detail.to_csv('../tmp/train/combine_detail.csv', index=False)
print('补充款型数量:', len(supplement_detail))
print('力洋匹配上款型数量:', len(liyang_detail))
print('汽车之家款型数量:', len(list(set(car_autohome_all.detail_slug.values))))
print('最终组合款型数量:', len(gpj_detail))

补充款型数量: 7258
力洋匹配上款型数量: 26785
汽车之家款型数量: 31437
最终组合款型数量: 34043


## 五.拷贝款型库到api包

In [170]:
def process_mum(df):
    text = df['mum']
    regex = re.compile("\(进口\)")
    if len(regex.findall(text)) != 0:
        text = '进口' + re.sub("\(进口\)", '', text)
    text = text.replace('・','')
    text = text.replace('-','')
    text = text.replace('一汽大众奥迪','一汽奥迪')
    text = text.replace('北京汽车','北汽')
    text = text.replace('上汽通用','通用')
    text = text.replace('北汽昌河','昌河汽车')
    text = text.replace('上汽大众','上海大众')
    text = text.replace('福田汽车','福田')
    text = text.replace('一汽海马','海马汽车')
    text = text.replace('Jeep','吉普')
    text = text.replace('进口路虎','路虎')
    text = text.replace('力帆汽车','重庆力帆')
    text = text.replace('领克汽车','领克')
    text = text.replace('东风启辰','东风日产')
    text = text.replace('蔚来汽车','蔚来')
    text = text.replace('吉利全球鹰','吉利康迪')
    
    return text

evaluation_predict_path = '/home/ml/ProgramProject/evaluation-predict/tmp/train/'

gpj_brand = pd.read_csv('../tmp/train/combine_brand.csv',low_memory=False)
gpj_model = pd.read_csv('../tmp/train/combine_model.csv',low_memory=False)
gpj_detail = pd.read_csv('../tmp/train/combine_detail.csv',low_memory=False)

gpj_model['mum'] = gpj_model.apply(process_mum, axis=1)
gpj_model.to_csv('../tmp/train/combine_model.csv', index=False)

gpj_brand.to_csv(evaluation_predict_path + 'combine_brand.csv', index=False)
gpj_model.to_csv(evaluation_predict_path + 'combine_model.csv', index=False)
gpj_detail.to_csv(evaluation_predict_path + 'combine_detail.csv', index=False)

## 六.新款型匹配

In [171]:
def process_year(df):
    if len(re.findall(r'(\d+)款',df['detail_model'])) > 0:
        return int(re.findall(r'(\d+)款',df['detail_model'])[0])
    else:
        return int(re.findall(r'\d\d\d\d',df['detail_model'])[0])

open_model_detail = pd.read_csv('../tmp/train/open_model_detail.csv', low_memory=False)
open_model_detail['year'] = open_model_detail.apply(process_year, axis=1)
open_model_detail = open_model_detail.rename(columns={'global_slug':'model_slug','year':'online_year'})
open_model_detail = open_model_detail.loc[(open_model_detail['price_bn'] > 0), :].reset_index(drop=True)
final_model = pd.read_csv('../tmp/train/final_liyang_model.csv')
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
old_car_autohome_all = car_autohome_all.loc[~(car_autohome_all['detail_slug'].isin(list(set(add_new_detail.detail_slug.values)))), :]

final_model = final_model.loc[(final_model['car_autohome_model_id'].notnull()), :].reset_index(drop=True)
gpj_match_model = open_model_detail.loc[(open_model_detail['model_slug'].isin(list(set(final_model.model_slug.values)))), :]
gpj_match_model = gpj_match_model.merge(final_model.loc[:,['model_slug','car_autohome_brand_id','car_autohome_model_id']],how='left',on=['model_slug'])
print('力洋总款型数:', len(open_model_detail))
print('力洋匹配的车系量:', len(final_model))
print('力洋匹配上车系的款型量:', len(gpj_match_model))

add_new_detail = pd.read_csv('../tmp/train/add_new_detail.csv')
add_new_detail = add_new_detail.drop(['manufacturer','body','energy'], axis=1)
print('汽车之家新增款型总量:',len(list(set(add_new_detail.detail_slug.values))))
print('汽车之家新增款型的车系总量:',len(list(set(add_new_detail.model_slug.values))))

# 汽车之家补充的车系
supplement_model = add_new_detail.loc[~(add_new_detail['model_slug'].isin(list(set(old_car_autohome_all.model_slug.values)))), :].reset_index(drop=True)
match_model = add_new_detail.loc[(add_new_detail['model_slug'].isin(list(set(old_car_autohome_all.model_slug.values)))), :].reset_index(drop=True)
# match_model = match_model.rename(columns={'model_slug':'car_autohome_model_id','detail_slug':'car_autohome_detail_id'}).loc[:, ['car_autohome_model_id','car_autohome_detail_id','detail_name','price_bn','online_year']]
print('汽车之家补充的车系量:', len(list(set(supplement_model.model_slug.values))))
print('汽车之家匹配的车系量:', len(list(set(match_model.model_slug.values))))
print('汽车之家匹配的款型量:', len(list(set(match_model.detail_slug.values))))

力洋总款型数: 38668
力洋匹配的车系量: 1550
力洋匹配上车系的款型量: 36511
汽车之家新增款型总量: 553
汽车之家新增款型的车系总量: 116
汽车之家补充的车系量: 25
汽车之家匹配的车系量: 91
汽车之家匹配的款型量: 445


In [172]:
def cal_div(df, car_autohome_temp):
    car_autohome_temp['price_bn_div'] = abs(df['price_bn'] - car_autohome_temp['price_bn'])
    temp = car_autohome_temp.loc[(car_autohome_temp['price_bn_div'] == min(list(car_autohome_temp.price_bn_div.values))),:].reset_index(drop=True)
    return pd.Series(list(temp.loc[0, ['detail_slug','detail_name','price_bn']].values))

model_year = add_new_detail.loc[:, ['model_slug','online_year']].drop_duplicates(['model_slug','online_year'])

result_model_year = pd.DataFrame()
no_have_model_year = pd.DataFrame()
except_model_year = pd.DataFrame()
count = 0
for car_autohome_model_id,online_year in model_year.loc[:, ['model_slug','online_year']].values:
    count = count+1
    gpj_temp = gpj_match_model.loc[(gpj_match_model['car_autohome_model_id'] == car_autohome_model_id)&(gpj_match_model['online_year'] == online_year), :].reset_index(drop=True)
    gpj_temp['car_autohome_detail_id'] = np.NAN
    gpj_temp['detail_name'] = np.NAN
    gpj_temp['car_autohome_price_bn'] = np.NAN
    car_autohome_temp = car_autohome_all.loc[(car_autohome_all['model_slug'] == car_autohome_model_id)&(car_autohome_all['online_year'] == online_year), :].reset_index(drop=True)
    if len(gpj_temp) == 0:
        except_model_year = except_model_year.append(car_autohome_temp, sort=False).reset_index(drop=True)
        continue
    gpj_temp[['car_autohome_detail_id','detail_name','car_autohome_price_bn']] = gpj_temp.apply(cal_div,args=(car_autohome_temp,),axis=1)
    gpj_temp['price_bn_rate'] = abs((gpj_temp['price_bn'] - gpj_temp['car_autohome_price_bn']) / gpj_temp['car_autohome_price_bn'])
    gpj_temp = gpj_temp.loc[(gpj_temp['price_bn_rate'] <= 0.05), :].reset_index(drop=True)
    if (len(gpj_temp)/len(car_autohome_temp)) >= 0.8:
        result_model_year = result_model_year.append(gpj_temp, sort=False).reset_index(drop=True)
    else:
        except_model_year = except_model_year.append(car_autohome_temp, sort=False).reset_index(drop=True)

result_model_year = result_model_year.drop(['car_autohome_brand_id','detail_name','car_autohome_price_bn','price_bn_rate'], axis=1)
except_model_year = except_model_year.drop(['price_bn_div'], axis=1)
print('使用力洋款型总量:',len(result_model_year))
print('使用汽车之家款型总量:',len(except_model_year))

使用力洋款型总量: 178
使用汽车之家款型总量: 609


In [173]:
# open_category = pd.read_csv('../tmp/train/open_category.csv', low_memory=False)
# gpj_brand = open_category.loc[(open_category['parent'].isnull()), :].reset_index(drop=True)
# gpj_model = open_category.loc[(open_category['parent'].notnull()), :].reset_index(drop=True)
# gpj_detail = pd.read_csv('../tmp/train/open_model_detail.csv', low_memory=False)
add_new_detail = pd.read_csv('../tmp/train/add_new_detail.csv')
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
old_car_autohome_all = car_autohome_all.loc[~(car_autohome_all['detail_slug'].isin(list(set(add_new_detail.detail_slug.values)))), :]

final_model = pd.read_csv('../tmp/train/final_liyang_model.csv')
combine_brand = pd.read_csv('../tmp/train/combine_brand.csv',low_memory=False)
combine_model = pd.read_csv('../tmp/train/combine_model.csv',low_memory=False)
combine_detail = pd.read_csv('../tmp/train/combine_detail.csv',low_memory=False)

# 补充品牌
def process_brand(df, column_name):
    return 'car_autohome_brand_'+str(df[column_name])

def process_model(df, column_name):
    return 'car_autohome_model_'+str(df[column_name])

def process_detail(df, column_name):
    return 'car_autohome_detail_'+str(df[column_name])

def process_keywords(df):
    p = Pinyin()
    keywords = p.get_pinyin(df['name'], '')
    final_keywords = ','+keywords+','+df['name']+','+keywords
    first_letter = p.get_initial(df['name'][0])
    return pd.Series([final_keywords,first_letter,keywords])

supplement_brand = list(set(add_new_detail.brand_slug.values).difference(set(old_car_autohome_all.brand_slug.values)))
supplement_brand = add_new_detail.loc[(add_new_detail['brand_slug'].isin(supplement_brand)), ['brand_slug','brand_name']]
supplement_brand = supplement_brand.drop_duplicates(['brand_slug','brand_name']).reset_index(drop=True)
supplement_brand = supplement_brand.rename(columns={'brand_name':'name','brand_slug':'slug'})
# supplement_brand['car_autohome_brand_id'] = supplement_brand['slug']
supplement_brand['slug'] = supplement_brand.apply(process_brand,args=('slug',), axis=1)
supplement_brand[['keywords', 'first_letter', 'pinyin']] = supplement_brand.apply(process_keywords, axis=1)

gpj_brand = pd.DataFrame([], columns=combine_brand.columns)
gpj_brand = gpj_brand.append(supplement_brand, sort=False)
gpj_brand['has_detailmodel'] = 1
gpj_brand['units'] = 0
gpj_brand['score'] = 0
gpj_brand['status'] = 'Y'
gpj_brand['popular'] = 'B'
gpj_brand['on_sale'] = '1'
gpj_brand.to_csv('../tmp/train/add_combine_brand.csv', index=False)
print('补充品牌数量:', len(supplement_brand))
print('汽车之家品牌数量:', len(list(set(add_new_detail.brand_slug.values))))

补充品牌数量: 4
汽车之家品牌数量: 67


In [174]:
# 补充车系
def process_brand_step2(df, final_model, gpj_brand):
    brand_id = final_model.loc[(final_model['car_autohome_brand_id'] == df['car_autohome_brand_id']), 'brand_id'].values
    if len(brand_id) == 0:
        return 'car_autohome_brand_'+str(df['car_autohome_brand_id'])
    slug = gpj_brand.loc[(gpj_brand['id'] == brand_id[0]), 'slug'].values[0]
    return slug
    
gpj_brand = combine_brand.append(gpj_brand, sort=False)
supplement_model = add_new_detail.loc[~(add_new_detail['model_slug'].isin(list(set(combine_model.car_autohome_model_id.values)))), :]
supplement_model_temp = supplement_model.loc[:, ['brand_slug','model_name','model_slug']]
supplement_model_temp = supplement_model_temp.drop_duplicates(['brand_slug','model_name','model_slug'])
supplement_model_temp = supplement_model_temp.rename(columns={'brand_slug':'parent', 'model_name':'name', 'model_slug':'slug'})
supplement_model_temp['car_autohome_model_id'] = supplement_model_temp['slug']
supplement_model_temp['slug'] = supplement_model_temp.apply(process_model,args=('slug',), axis=1)
supplement_model_temp[['keywords', 'first_letter', 'pinyin']] = supplement_model_temp.apply(process_keywords, axis=1)
supplement_model_temp = supplement_model_temp.drop(['first_letter'], axis=1)
merge = add_new_detail.loc[:, ['model_slug', 'body', 'manufacturer']].drop_duplicates(['model_slug', 'body', 'manufacturer']).rename(columns={'model_slug':'car_autohome_model_id','body':'classified','manufacturer':'mum'}).reset_index(drop=True)
supplement_model_temp = supplement_model_temp.merge(merge, how='left', on=['car_autohome_model_id'])
supplement_model_temp['normalized_name'] = supplement_model_temp['name']

gpj_model = pd.DataFrame([], columns=combine_model.columns)
gpj_model = gpj_model.append(supplement_model_temp, sort=False)
gpj_model['car_autohome_model_id'] = gpj_model['car_autohome_model_id'].astype(int)
car_autohome_model = add_new_detail.loc[:,['brand_slug','model_slug']].drop_duplicates(['brand_slug','model_slug']).rename(columns={'model_slug':'car_autohome_model_id','brand_slug':'car_autohome_brand_id'}).reset_index(drop=True)
gpj_model = gpj_model.merge(car_autohome_model, how='left', on=['car_autohome_model_id'])
gpj_model['parent'] = gpj_model.apply(process_brand_step2,args=(final_model, gpj_brand), axis=1)
gpj_model = gpj_model.drop(['car_autohome_brand_id'], axis=1)
gpj_model['has_detailmodel'] = 1
gpj_model['units'] = 0
gpj_model['score'] = 0
gpj_model['status'] = 'Y'
gpj_model['popular'] = 'B'
gpj_model['on_sale'] = '1'
gpj_model.to_csv('../tmp/train/add_combine_model.csv', index=False)
print('补充车系数量:', len(list(set(supplement_model_temp.slug.values))))
print('汽车之家车系数量:', len(list(set(add_new_detail.model_slug.values))))

补充车系数量: 25
汽车之家车系数量: 116


In [175]:
# 补充款型
def process_model_step2(df, gpj_model, column_name):
    return gpj_model.loc[(gpj_model['car_autohome_model_id'] == df['model_slug']), 'slug'].values[0]

supplement_detail = supplement_model.append(except_model_year, sort=False).reset_index(drop=True)
      
liyang_detail = result_model_year.copy()
gpj_model = combine_model.append(gpj_model, sort=False)
liyang_detail = liyang_detail.drop(['car_autohome_model_id'], axis=1)
supplement_detail = car_autohome_all.loc[(car_autohome_all['detail_slug'].isin(list(set(supplement_detail.detail_slug.values)))), :]
supplement_detail = supplement_detail.loc[:, ['model_slug','detail_slug','detail_name','price_bn','online_year','volume','control','volume_extend','emission_standard','listed_year']]
supplement_detail = supplement_detail.rename(columns={'detail_slug':'car_autohome_detail_id',
                                                      'detail_name':'detail_model'})
supplement_detail['model_slug'] = supplement_detail.apply(process_model_step2, args=(gpj_model, 'model_slug',), axis=1)
supplement_detail['model_detail_slug'] = supplement_detail.apply(process_detail, args=('car_autohome_detail_id',), axis=1)

gpj_detail = liyang_detail.append(supplement_detail, sort=False)
gpj_detail = gpj_detail.rename(columns={'model_slug':'global_slug','online_year':'year','model_detail_slug':'detail_model_slug'})
gpj_detail['created_on'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
gpj_detail['status'] = 'Y'
gpj_detail['has_param'] = 'Y'
gpj_detail.to_csv('../tmp/train/add_combine_detail.csv', index=False)
print('补充款型数量:', len(supplement_detail))
print('力洋匹配上款型数量:', len(liyang_detail))
print('汽车之家款型数量:', len(list(set(car_autohome_all.detail_slug.values))))

补充款型数量: 609
力洋匹配上款型数量: 178
汽车之家款型数量: 31990


## 七.拷贝款型库到api包

In [176]:
def process_mum(df):
    text = df['mum']
    regex = re.compile("\(进口\)")
    if len(regex.findall(text)) != 0:
        text = '进口' + re.sub("\(进口\)", '', text)
    text = text.replace('・','')
    text = text.replace('-','')
    text = text.replace('一汽大众奥迪','一汽奥迪')
    text = text.replace('北京汽车','北汽')
    text = text.replace('上汽通用','通用')
    text = text.replace('北汽昌河','昌河汽车')
    text = text.replace('上汽大众','上海大众')
    text = text.replace('福田汽车','福田')
    text = text.replace('一汽海马','海马汽车')
    text = text.replace('Jeep','吉普')
    text = text.replace('进口路虎','路虎')
    text = text.replace('力帆汽车','重庆力帆')
    text = text.replace('领克汽车','领克')
    text = text.replace('东风启辰','东风日产')
    text = text.replace('蔚来汽车','蔚来')
    text = text.replace('吉利全球鹰','吉利康迪')
    
    return text

evaluation_predict_path = '/home/ml/ProgramProject/evaluation-predict/tmp/train/'

gpj_brand = pd.read_csv('../tmp/train/combine_brand.csv',low_memory=False)
gpj_model = pd.read_csv('../tmp/train/combine_model.csv',low_memory=False)
gpj_detail = pd.read_csv('../tmp/train/combine_detail.csv',low_memory=False)
print('当前品牌数:', len(gpj_brand))
print('当前车系数:', len(gpj_model))
print('当前款型数:', len(gpj_detail))
add_combine_brand = pd.read_csv('../tmp/train/add_combine_brand.csv')
add_combine_model = pd.read_csv('../tmp/train/add_combine_model.csv')
add_combine_detail = pd.read_csv('../tmp/train/add_combine_detail.csv')

add_combine_model['mum'] = add_combine_model.apply(process_mum, axis=1)
gpj_detail = gpj_detail.loc[~(gpj_detail['car_autohome_detail_id'].isin(list(set(add_combine_detail.car_autohome_detail_id.values)))), :]

gpj_brand = gpj_brand.append(add_combine_brand, sort=False).reset_index(drop=True)
gpj_model = gpj_model.append(add_combine_model, sort=False).reset_index(drop=True)
gpj_detail = gpj_detail.append(add_combine_detail, sort=False).reset_index(drop=True)

print('新增之后品牌数:', len(gpj_brand))
print('新增之后车系数:', len(gpj_model))
print('新增之后款型数:', len(gpj_detail))
gpj_brand.to_csv('../tmp/train/combine_brand.csv', index=False)
gpj_model.to_csv('../tmp/train/combine_model.csv', index=False)
gpj_detail.to_csv('../tmp/train/combine_detail.csv', index=False)

gpj_brand.to_csv(evaluation_predict_path + 'combine_brand.csv', index=False)
gpj_model.to_csv(evaluation_predict_path + 'combine_model.csv', index=False)
gpj_detail.to_csv(evaluation_predict_path + 'combine_detail.csv', index=False)

add_combine_brand.to_csv(evaluation_predict_path + 'add_combine_brand.csv', index=False)
add_combine_model.to_csv(evaluation_predict_path + 'add_combine_model.csv', index=False)
add_combine_detail.to_csv(evaluation_predict_path + 'add_combine_detail.csv', index=False)

当前品牌数: 178
当前车系数: 1826
当前款型数: 34043
新增之后品牌数: 182
新增之后车系数: 1851
新增之后款型数: 34582


In [177]:
brand = gpj_brand.loc[:, ['name','slug']].rename(columns={'name':'brand_name','slug':'parent'})
model = gpj_model.loc[:, ['name','slug','parent']].rename(columns={'name':'model_name','slug':'global_slug'})
model = model.merge(brand, how='left', on=['parent'])
final = gpj_detail.merge(model,how='left',on=['global_slug'])
final = final.sort_values(by=['parent','global_slug','year','price_bn'])
final.to_csv('../tmp/train/man.csv')
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34582 entries, 33476 to 29157
Data columns (total 29 columns):
id                        26831 non-null float64
source_id                 15019 non-null float64
checker_runtime_id        13245 non-null float64
old_dmodel                17752 non-null object
detail_model              34582 non-null object
detail_model_slug         34582 non-null object
price_bn                  34582 non-null float64
cont_vprice               22469 non-null float64
url                       13245 non-null object
global_slug               34582 non-null object
domain                    26831 non-null object
status                    34582 non-null object
year                      34582 non-null int64
has_param                 34582 non-null object
volume                    34582 non-null float64
vv                        22279 non-null float64
listed_year               34582 non-null int64
delisted_year             12241 non-null float64
control          